<a href="https://colab.research.google.com/github/HuyenNguyenHelen/CORD-19-KG/blob/master/covid_19_kg_mining_MetamapEnt_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Preliminaries

In [1]:
# If using Google Colab run this cell 

# select tensorflow version for colab 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf 

print('TensorFlow  version: {}'.format(tf.__version__))

# Get the GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow  version: 1.15.2
Found GPU at: /device:GPU:0


In [3]:
%%capture 
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

In [4]:
# All imports used in this tutorial 
%tensorflow_version 1.x
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model

def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


# 2. Loading a Knowledge Graph dataset

In [9]:
from ampligraph.datasets import load_fb15k_237, load_wn18rr, load_yago3_10
import io
import requests

In [10]:
# Downloading the csv file from your GitHub account
re_pdf = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/new_triples_with_predefined_relations_pdf_June20_meta-only.csv').content
re_pmc = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/new_triples_with_predefined_relations_pmc_June20_meta-only.csv').content
pdf = pd.read_csv(io.StringIO(re_pdf.decode('utf-8')))
pmc =  pd.read_csv(io.StringIO(re_pmc.decode('utf-8')))
dataset = pd.concat([pmc, pdf])
print(dataset.columns)
dataset.drop(columns='Unnamed: 0', inplace=True)
dataset = dataset[['subject','new_relation', 'object' ]]
dataset.head(5)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'subject', 'object', 'new_relation'], dtype='object')


,subject,new_relation,object
0,step,gene_gene,endosomes
1,disease,disease_disease,pneumonia
2,prophylaxis,treat_procedure_gene,end
3,prophylaxis,treat_procedure_gene,end
4,impact,gene_gene,past


In [20]:
dataset[dataset['subject']=='coronavirus']

,subject,new_relation,object
57,coronavirus,disease_disease,mouse hepatitis virus
387,coronavirus,disease_disease,kobuvirus
577,coronavirus,disease_disease,severe acute respiratory syndrome
578,coronavirus,disease_disease,syndrome
583,coronavirus,disease_disease,renal failure
...,...,...,...
46540,coronavirus,disease_disease,liver
46546,coronavirus,disease_disease,syndrome
46547,coronavirus,disease_disease,syndrome
46548,coronavirus,disease_disease,severe acute respiratory syndrome


In [11]:
print('Total triples in the KG:', dataset.shape)

Total triples in the KG: (59070, 3)


In [12]:
print('the number of relations: \n', dataset['new_relation'].value_counts())

the number of relations: 
 disease_disease                    18995
treat_procedure_treat_procedure     5528
gene_gene                           4366
disease_treat_procedure             4175
treat_procedure_disease             3894
symptom_symptom                     3772
disease_gene                        3397
disease_symptom                     2742
gene_disease                        2573
gene_treat_procedure                2276
symptom_disease                     2260
treat_procedure_gene                2191
gene_symptom                         526
treat_procedure_symptom              466
symptom_treat_procedure              406
disease_species                      348
symptom_gene                         235
drug_treat_procedure                 183
drug_disease                         153
gene_species                         149
treat_procedure_species              125
treat_procedure_drug                  81
drug_gene                             78
disease_drug                  

## 2.1 Create training, validation and test splits

In [13]:
from ampligraph.evaluation import train_test_split_no_unseen
# get the validation set of size 500
test_train, X_valid = train_test_split_no_unseen(dataset.values, 500, seed=0)

# get the test set of size 1000 from the remaining triples
X_train, X_test = train_test_split_no_unseen(test_train, 1000, seed=0)

print('Total triples:', dataset.shape)
print('Size of train:', X_train.shape)
print('Size of valid:', X_valid.shape)
print('Size of test:', X_test.shape)

Total triples: (59070, 3)
Size of train: (57570, 3)
Size of valid: (500, 3)
Size of test: (1000, 3)


# 3. Model Training

In [14]:
from ampligraph.latent_features import TransE

# Train a KGE model
model = TransE(k=300, 
               epochs=100, 
               eta=1, 
               loss='multiclass_nll', 
               initializer='xavier', initializer_params={'uniform': False},
               regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},
               optimizer= 'adam', optimizer_params= {'lr': 0.0001}, 
               seed= 0, batches_count= 10, verbose=True)

model.fit(X_train)
# ----------------------
# Evaluate: 
# Filtered evaluation with ranking strategy assigning worst rank to break ties

from ampligraph.utils import save_model, restore_model
save_model(model, 'TransE.pkl')
model = restore_model('TransE.pkl')

# create the filter 
X_filter = np.concatenate([X_train, X_valid, X_test], 0)

# compute ranks
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

# ranks are computed per triple
print('Test set:', X_test.shape)
print('Size of ranks:', ranks.shape)

# Aggregate metrics show the aggregate performance of the model on the test set using a single number
display_aggregate_metrics(ranks)
# ----------------------

100%|██████████| 1000/1000 [00:03<00:00, 274.01it/s]


Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 386.886
Mean Reciprocal Rank: 0.23319439198377404
Hits@1: 0.1595
Hits@10: 0.381
Hits@100: 0.6625


# 4. Knowledge Discovery

### Relations

In [15]:
# Relations
'''
disease_disease                    87479
gene_gene                          25101
disease_species                    21152
gene_disease                       17414
disease_gene                       17383
treat_procedure_disease            15221
disease_treat_procedure            14696
treat_procedure_treat_procedure    13533
treat_procedure_gene                8913
gene_treat_procedure                8913
treat_procedure_species             5307
gene_species                        3894
disease_symptom                     2501
symptom_disease                     1409
treat_procedure_symptom              391
gene_symptom                         310
symptom_species                      253
symptom_treat_procedure              211
symptom_gene                         170
symptom_symptom                      141
'''

'\ndisease_disease                    87479\ngene_gene                          25101\ndisease_species                    21152\ngene_disease                       17414\ndisease_gene                       17383\ntreat_procedure_disease            15221\ndisease_treat_procedure            14696\ntreat_procedure_treat_procedure    13533\ntreat_procedure_gene                8913\ngene_treat_procedure                8913\ntreat_procedure_species             5307\ngene_species                        3894\ndisease_symptom                     2501\nsymptom_disease                     1409\ntreat_procedure_symptom              391\ngene_symptom                         310\nsymptom_species                      253\nsymptom_treat_procedure              211\nsymptom_gene                         170\nsymptom_symptom                      141\n'

##  Question - Answering

``` 
    <head, relation, ?> 
    <head, ?,        tail>
    <?,    relation, tail>
```

### Query: what are the possible therapeutic procedure of covid-19??

In [21]:
# Query: what are the possible therapeutic procedure of covid-19??

from ampligraph.discovery import query_topn

# restore the previously saved model to save time
model = restore_model('TransE.pkl')

triples, scores = query_topn(model, top_n=20, 
                             head='coronavirus', 
                             relation='disease_treat_procedure', 
                             tail=None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.11337947845459 	 ['coronavirus' 'disease_treat_procedure' 'detected'] 
Score: -10.829673767089844 	 ['coronavirus' 'disease_treat_procedure' 'destruction'] 
Score: -11.142451286315918 	 ['coronavirus' 'disease_treat_procedure' 'mechanical ventilation'] 
Score: -11.163158416748047 	 ['coronavirus' 'disease_treat_procedure' 'elevation'] 
Score: -11.16749382019043 	 ['coronavirus' 'disease_treat_procedure' 'clearance'] 
Score: -11.181879043579102 	 ['coronavirus' 'disease_treat_procedure' 'release'] 
Score: -11.247224807739258 	 ['coronavirus' 'disease_treat_procedure' 'expression'] 
Score: -11.263041496276855 	 ['coronavirus' 'disease_treat_procedure' 'presence'] 
Score: -11.357131958007812 	 ['coronavirus' 'disease_treat_procedure' 'detection'] 
Score: -11.363958358764648 	 ['coronavirus' 'disease_treat_procedure' 'induction'] 
Score: -11.469465255737305 	 ['coronavirus' 'disease_treat_procedure' 'quarantine'] 
Score: -11.522125244140625 	 ['coronavirus' 'disease_treat_proced

In [22]:
# Query: what are the possible therapeutic procedure of covid-19??

from ampligraph.discovery import query_topn

# restore the previously saved model to save time
model = restore_model('TransE.pkl')

triples, scores = query_topn(model, top_n=20, 
                             head=None, 
                             relation='treat_procedure_disease', 
                             tail='coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.722274780273438 	 ['administration' 'treat_procedure_disease' 'coronavirus'] 
Score: -10.930908203125 	 ['inoculation' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.050116539001465 	 ['immunization' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.142293930053711 	 ['expression' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.151449203491211 	 ['isolation' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.154409408569336 	 ['reduction' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.167686462402344 	 ['therapy' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.179778099060059 	 ['presence' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.205928802490234 	 ['application' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.253530502319336 	 ['influenza vaccination' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.35280704498291 	 ['infiltration' 'treat_procedure_disease' 'coronavirus'] 
Score: -11.402839660644531 	 ['vaccination' 'treat_proc

### Query: What are related diseases of Covid-19

In [23]:
# Predict relation

triples, scores = query_topn(model, top_n=20, 
                             head='coronavirus', 
                             relation='disease_disease', 
                             tail= None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -13.101186752319336 	 ['coronavirus' 'disease_disease' 'human metapneumovirus'] 
Score: -13.128988265991211 	 ['coronavirus' 'disease_disease' 'bat coronavirus'] 
Score: -13.156639099121094 	 ['coronavirus' 'disease_disease' 'febrile illness'] 
Score: -13.279833793640137 	 ['coronavirus' 'disease_disease' 'acute respiratory tract infection'] 
Score: -13.337406158447266 	 ['coronavirus' 'disease_disease' 'respiratory tract disease'] 
Score: -13.427349090576172 	 ['coronavirus' 'disease_disease' 'hepatic necrosis'] 
Score: -13.551551818847656 	 ['coronavirus' 'disease_disease' 'contagious disease'] 
Score: -13.66872501373291 	 ['coronavirus' 'disease_disease' 'viral disease'] 
Score: -13.688700675964355 	 ['coronavirus' 'disease_disease' 'kidney failure'] 
Score: -13.695825576782227 	 ['coronavirus' 'disease_disease' 'lower respiratory tract infection'] 
Score: -13.723230361938477 	 ['coronavirus' 'disease_disease' 'severe acute respiratory syndrome'] 
Score: -13.760993003845215 	

In [24]:
# Predict relation

triples, scores = query_topn(model, top_n=20, 
                             head=None, 
                             relation='disease_disease', 
                             tail= 'coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -13.22624397277832 	 ['reovirus' 'disease_disease' 'coronavirus'] 
Score: -13.256596565246582 	 ['calicivirus' 'disease_disease' 'coronavirus'] 
Score: -13.46108627319336 	 ['mucopurulent conjunctivitis' 'disease_disease' 'coronavirus'] 
Score: -13.461867332458496 	 ['he' 'disease_disease' 'coronavirus'] 
Score: -13.741909980773926 	 ['normocytic anemia' 'disease_disease' 'coronavirus'] 
Score: -13.764816284179688 	 ['virus infection' 'disease_disease' 'coronavirus'] 
Score: -13.825664520263672 	 ['bronchiolitis obliterans' 'disease_disease' 'coronavirus'] 
Score: -13.83259391784668 	 ['deltacoronavirus' 'disease_disease' 'coronavirus'] 
Score: -13.838094711303711 	 ['autoimmune hemolytic anemia' 'disease_disease' 'coronavirus'] 
Score: -13.922308921813965 	 ['respiratory virus' 'disease_disease' 'coronavirus'] 
Score: -13.954536437988281 	 ['tuberculosis' 'disease_disease' 'coronavirus'] 
Score: -13.995033264160156 	 ['fungal infection' 'disease_disease' 'coronavirus'] 
Score: 

### Query: What are the genes/proteins related to Covid-19

In [25]:
# Predict relation

triples, scores = query_topn(model, top_n=25, 
                             head='coronavirus', 
                             relation='disease_gene', 
                             tail=None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.575424194335938 	 ['coronavirus' 'disease_gene' 'damage'] 
Score: -10.588289260864258 	 ['coronavirus' 'disease_gene' 'rare'] 
Score: -10.624994277954102 	 ['coronavirus' 'disease_gene' 'cell'] 
Score: -10.67475700378418 	 ['coronavirus' 'disease_gene' 'body'] 
Score: -10.783074378967285 	 ['coronavirus' 'disease_gene' 'air'] 
Score: -10.804056167602539 	 ['coronavirus' 'disease_gene' '16'] 
Score: -10.921653747558594 	 ['coronavirus' 'disease_gene' 'set'] 
Score: -10.932303428649902 	 ['coronavirus' 'disease_gene' 'genome'] 
Score: -10.934850692749023 	 ['coronavirus' 'disease_gene' 'large'] 
Score: -10.97941780090332 	 ['coronavirus' 'disease_gene' 'host cell'] 
Score: -11.019573211669922 	 ['coronavirus' 'disease_gene' 'mask'] 
Score: -11.050115585327148 	 ['coronavirus' 'disease_gene' 'type'] 
Score: -11.151134490966797 	 ['coronavirus' 'disease_gene' 'end'] 
Score: -11.156850814819336 	 ['coronavirus' 'disease_gene' 'hepatocytes'] 
Score: -11.242757797241211 	 ['coronav

In [26]:
# Predict relation

triples, scores = query_topn(model, top_n=20, 
                             head= None, 
                             relation='gene_disease', 
                             tail='coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -11.049840927124023 	 ['all' 'gene_disease' 'coronavirus'] 
Score: -11.282011032104492 	 ['she' 'gene_disease' 'coronavirus'] 
Score: -11.325490951538086 	 ['cat' 'gene_disease' 'coronavirus'] 
Score: -11.33282470703125 	 ['gene' 'gene_disease' 'coronavirus'] 
Score: -11.393682479858398 	 ['aim' 'gene_disease' 'coronavirus'] 
Score: -11.51169204711914 	 ['light' 'gene_disease' 'coronavirus'] 
Score: -11.579887390136719 	 ['sex' 'gene_disease' 'coronavirus'] 
Score: -11.691695213317871 	 ['rest' 'gene_disease' 'coronavirus'] 
Score: -11.747001647949219 	 ['protein' 'gene_disease' 'coronavirus'] 
Score: -11.851118087768555 	 ['damage' 'gene_disease' 'coronavirus'] 
Score: -11.85301685333252 	 ['pig' 'gene_disease' 'coronavirus'] 
Score: -11.883543014526367 	 ['impact' 'gene_disease' 'coronavirus'] 
Score: -11.902676582336426 	 ['type' 'gene_disease' 'coronavirus'] 
Score: -11.935670852661133 	 ['microglia' 'gene_disease' 'coronavirus'] 
Score: -11.99493408203125 	 ['mix' 'gene_dis

### Query: What species transmit the covid-19

In [29]:
# Predict relation
triples, scores = query_topn(model, top_n=20, 
                             tail=None, 
                             relation='disease_species', 
                             head='coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -9.288200378417969 	 ['coronavirus' 'disease_species' 'specie'] 
Score: -10.098377227783203 	 ['coronavirus' 'disease_species' 'livestock'] 
Score: -10.185588836669922 	 ['coronavirus' 'disease_species' 'animal'] 
Score: -10.312093734741211 	 ['coronavirus' 'disease_species' 'vector'] 
Score: -10.668992042541504 	 ['coronavirus' 'disease_species' 'animal model'] 
Score: -10.89625072479248 	 ['coronavirus' 'disease_species' 'working'] 
Score: -10.973854064941406 	 ['coronavirus' 'disease_species' 'meat'] 
Score: -11.993785858154297 	 ['coronavirus' 'disease_species' 'show'] 
Score: -12.48626708984375 	 ['coronavirus' 'disease_species' 'pet'] 
Score: -12.537677764892578 	 ['coronavirus' 'disease_species' 'experimental animal model'] 
Score: -12.714065551757812 	 ['coronavirus' 'disease_species' 'animal kingdom'] 
Score: -12.84627914428711 	 ['coronavirus' 'disease_species' 'thoracotomy'] 
Score: -12.9278564453125 	 ['coronavirus' 'disease_species' 'cor pulmonale'] 
Score: -12.9341

### Query: what are symptoms of the covid-19

In [30]:

triples, scores = query_topn(model, top_n=20, 
                             head=None, 
                             relation='symptom_disease', 
                             tail='coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.903974533081055 	 ['fever' 'symptom_disease' 'coronavirus'] 
Score: -11.251813888549805 	 ['finding' 'symptom_disease' 'coronavirus'] 
Score: -11.260201454162598 	 ['pain' 'symptom_disease' 'coronavirus'] 
Score: -11.329851150512695 	 ['cough' 'symptom_disease' 'coronavirus'] 
Score: -11.337830543518066 	 ['diarrhea' 'symptom_disease' 'coronavirus'] 
Score: -11.344078063964844 	 ['diarrhoea' 'symptom_disease' 'coronavirus'] 
Score: -11.375017166137695 	 ['headache' 'symptom_disease' 'coronavirus'] 
Score: -11.386722564697266 	 ['illness' 'symptom_disease' 'coronavirus'] 
Score: -11.449350357055664 	 ['weakness' 'symptom_disease' 'coronavirus'] 
Score: -11.482715606689453 	 ['dyspnea' 'symptom_disease' 'coronavirus'] 
Score: -11.487443923950195 	 ['lethargy' 'symptom_disease' 'coronavirus'] 
Score: -11.492292404174805 	 ['watery diarrhea' 'symptom_disease' 'coronavirus'] 
Score: -11.53415298461914 	 ['malaise' 'symptom_disease' 'coronavirus'] 
Score: -11.568134307861328 	 ['d

In [31]:
triples, scores = query_topn(model, top_n=20, 
                             tail=None, 
                             relation='disease_symptom', 
                             head='coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.91246223449707 	 ['coronavirus' 'disease_symptom' 'illness'] 
Score: -10.998165130615234 	 ['coronavirus' 'disease_symptom' 'diarrhea'] 
Score: -11.126043319702148 	 ['coronavirus' 'disease_symptom' 'organ failure'] 
Score: -11.241591453552246 	 ['coronavirus' 'disease_symptom' 'severe diarrhea'] 
Score: -11.34196662902832 	 ['coronavirus' 'disease_symptom' 'fever'] 
Score: -11.36102294921875 	 ['coronavirus' 'disease_symptom' 'symptom'] 
Score: -11.391937255859375 	 ['coronavirus' 'disease_symptom' 'coughing'] 
Score: -11.428824424743652 	 ['coronavirus' 'disease_symptom' 'nasal discharge'] 
Score: -11.528543472290039 	 ['coronavirus' 'disease_symptom' 'cough'] 
Score: -11.631805419921875 	 ['coronavirus' 'disease_symptom' 'respiratory distress'] 
Score: -11.722046852111816 	 ['coronavirus' 'disease_symptom' 'fatigue'] 
Score: -11.725456237792969 	 ['coronavirus' 'disease_symptom' 'dyspnea'] 
Score: -11.734771728515625 	 ['coronavirus' 'disease_symptom' 'nausea'] 
Score: -1

In [ ]:
# Compare the raw call and graph embedding results for the symptoms query

## raw call
#dataset.loc[(dataset['subject']=='symptom')].head(20)

#####==> the result of raw call seems to be better

## Query: Potential Drug

In [32]:

triples, scores = query_topn(model, top_n=30, 
                             head=None, 
                             relation='drug_disease', 
                             tail='coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.676704406738281 	 ['drug' 'drug_disease' 'coronavirus'] 
Score: -11.033642768859863 	 ['monolayers' 'drug_disease' 'coronavirus'] 
Score: -11.795158386230469 	 ['azithromycin' 'drug_disease' 'coronavirus'] 
Score: -12.091232299804688 	 ['monoclonal antibody' 'drug_disease' 'coronavirus'] 
Score: -12.096012115478516 	 ['fluconazole' 'drug_disease' 'coronavirus'] 
Score: -13.230453491210938 	 ['impaction' 'drug_disease' 'coronavirus'] 
Score: -13.257286071777344 	 ['basiliximab' 'drug_disease' 'coronavirus'] 
Score: -13.307207107543945 	 ['tea' 'drug_disease' 'coronavirus'] 
Score: -13.329961776733398 	 ['bocavirus' 'drug_disease' 'coronavirus'] 
Score: -13.369889259338379 	 ['syphilis' 'drug_disease' 'coronavirus'] 
Score: -13.384997367858887 	 ['right ventricular impairment' 'drug_disease' 'coronavirus'] 
Score: -13.39857006072998 	 ['retinitis' 'drug_disease' 'coronavirus'] 
Score: -13.400906562805176 	 ['common cold' 'drug_disease' 'coronavirus'] 
Score: -13.40708160400390

In [34]:

triples, scores = query_topn(model, top_n=30, 
                             tail=None, 
                             relation='disease_drug', 
                             head='coronavirus', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -12.000625610351562 	 ['coronavirus' 'disease_drug' 'monoclonal antibody'] 
Score: -12.196426391601562 	 ['coronavirus' 'disease_drug' 'drug'] 
Score: -12.353361129760742 	 ['coronavirus' 'disease_drug' 'azithromycin'] 
Score: -12.447586059570312 	 ['coronavirus' 'disease_drug' 'monolayer'] 
Score: -12.935935974121094 	 ['coronavirus' 'disease_drug' 'monolayers'] 
Score: -13.008323669433594 	 ['coronavirus' 'disease_drug' 'azathioprine 75 mg'] 
Score: -13.153951644897461 	 ['coronavirus' 'disease_drug' 'human virus'] 
Score: -13.177911758422852 	 ['coronavirus' 'disease_drug' 'influenza virus'] 
Score: -13.182831764221191 	 ['coronavirus' 'disease_drug' 'emphysema'] 
Score: -13.193558692932129 	 ['coronavirus' 'disease_drug' 'hepatitis virus'] 
Score: -13.208952903747559 	 ['coronavirus' 'disease_drug' 'acinus'] 
Score: -13.228562355041504 	 ['coronavirus' 'disease_drug' 'cryoablation'] 
Score: -13.243635177612305 	 ['coronavirus' 'disease_drug' 'severe acute respiratory syndrom